## Stress tests
Stress Test 1: The client makes the same request very quickly min (10000 times).

Stress Test 2: Two or more clients make the possible requests randomly (10000 times).

Stress Test 3: Immediate occupancy of all seats/reservations by 2 clients. - Idea is we have one pool for reservation and 2 clients want to claim as much as possible. A situation where one client claims all is undesirable.

Stress Test 4: Constant cancellations and seat occupancy. (For same seat 10000 times)

Stress Test 5: Update of 1000 reservation. (For example change date for all reservation)

In [ ]:
import api, setup
import uuid
import random
import threading
import time

In [ ]:
setup()

In [ ]:
num_repetitions = 10000

In [ ]:
def stress_1():
    user_id = uuid.uuid4()
    book_id = uuid.uuid4()

    num_reservations = 0
    for i in range(num_repetitions):
        reservation = api.make_reservation({user_id, book_id})
        if reservation:
            num_reservations += 1
            print(f"Reservation {reservation.id} made")

    api.delete_all_reservations()
    print(f"Made {num_reservations} reservations")


In [ ]:
def stress_2():
    user_id1 = uuid.uuid4()
    user_id2 = uuid.uuid4()
    book_ids = [uuid.uuid4() for _ in range(1000)]

    successful_reservations = {user_id1: 0, user_id2: 0}
    unsuccessful_reservations = {user_id1: 0, user_id2: 0}

    def make_reservation(user_id):
        for i in range(num_repetitions):
            book_id = random.choice(book_ids)
            reservation = api.make_reservation({user_id, book_id})
            if reservation:
                successful_reservations[user_id] += 1
            else:
                unsuccessful_reservations[user_id] += 1

    threads = []
    for user_id in [user_id1, user_id2]:
        t = threading.Thread(target = make_reservation, args = (user_id,))
        t.start()
        threads.append(t)
        print(f"Thread for user {user_id} started")

    for t in threads:
        t.join()

    api.delete_all_reservations()
    print(f"User 1 made {successful_reservations[user_id1]} successful reservations and {unsuccessful_reservations[user_id1]} unsuccessful reservations")
    print(f"User 2 made {successful_reservations[user_id2]} successful reservations and {unsuccessful_reservations[user_id2]} unsuccessful reservations")

            


In [ ]:
def stress_3():
    user_id1 = uuid.uuid4()
    user_id2 = uuid.uuid4()
    book_ids = [uuid.uuid4() for _ in range(1000)]

    successful_reservations = {user_id1: 0, user_id2: 0}
    unsuccessful_reservations = {user_id1: 0, user_id2: 0}

    def make_reservation(user_id):
        for i in range(1000):
            reservation = api.make_reservation({user_id, book_ids[i]})
            if reservation:
                successful_reservations[user_id] += 1
                time.sleep(0.2)
            else:
                unsuccessful_reservations[user_id] += 1

    threads = []
    for user_id in [user_id1, user_id2]:
        t = threading.Thread(target = make_reservation, args = (user_id,))
        t.start()
        threads.append(t)
        print(f"Thread for user {user_id} started")

    for t in threads:
        t.join()

    api.delete_all_reservations()
    print(f"User 1 made {successful_reservations[user_id1]} successful reservations and {unsuccessful_reservations[user_id1]} unsuccessful reservations")
    print(f"User 2 made {successful_reservations[user_id2]} successful reservations and {unsuccessful_reservations[user_id2]} unsuccessful reservations")

    

In [ ]:
def stress_4():
    user_id1 = uuid.uuid4()
    user_id2 = uuid.uuid4()
    book_id = uuid.uuid4()

    successful_reservations = {user_id1: 0, user_id2: 0}
    unsuccessful_reservations = {user_id1: 0, user_id2: 0}

    def make_reservation(user_id):
        for i in range(num_repetitions):
            reservation = api.make_reservation({user_id, book_id})
            if reservation:
                successful_reservations[user_id] += 1
                time.sleep(random.uniform(0.05, 0.3))
                api.delete_reservation(book_id)
            else:
                unsuccessful_reservations[user_id] += 1
    
    threads = []
    for user_id in [user_id1, user_id2]:
        t = threading.Thread(target = make_reservation, args = (user_id,))
        t.start()
        threads.append(t)
        print(f"Thread for user {user_id} started")

    for t in threads:
        t.join()


    api.delete_all_reservations()
    print(f"User 1 made {successful_reservations[user_id1]} successful reservations and {unsuccessful_reservations[user_id1]} unsuccessful reservations")
    print(f"User 2 made {successful_reservations[user_id2]} successful reservations and {unsuccessful_reservations[user_id2]} unsuccessful reservations")


In [ ]:
def stress_5():
    user_id = uuid.uuid4()
    book_id = uuid.uuid4()

    api.make_reservation({user_id, book_id})

    successful_updates = 0

    for i in range(num_repetitions):
        new_user_id = uuid.uuid4()
        new_book_id = uuid.uuid4()
        update = api.update_reservation({book_id, new_user_id, new_book_id})
        if update:
            successful_updates += 1
            book_id = new_book_id

    api.delete_all_reservations()
    print(f"Made {successful_updates} updates")